In [1]:
import mocalum as mc
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

In [2]:
# create mocalum object
pl_mc = mc.Mocalum()

In [3]:
# add a lidar to the object with description of uncertainties
uncertainty = {
    'unc_est' : 0.1,
    'unc_rng' : 10,
    'unc_az' : 0.1,
    'unc_el' : 0.1,
    'corr_coef' : 0,

 }


# whittle_xyz = np.array([0, 500, 0])
# koshava_xyz = np.array([0, 0, 0])
# sterenn_xyz = np.array([500, 0, 0])

whittle_xyz = np.array([0, 500, 0])
koshava_xyz = np.array([500, -500, 0])
sterenn_xyz = np.array([-500, -500, 0])

# single line setup for triple Doppler are to be avoided!
whittle_xyz = np.array([0, 500, 0])
koshava_xyz = np.array([10, 0, 0])
sterenn_xyz = np.array([0, -500, 0])



pl_mc.add_lidar('koshava', koshava_xyz, uncertainty)
pl_mc.add_lidar('sterenn', sterenn_xyz, uncertainty)
pl_mc.add_lidar('whittle', whittle_xyz, uncertainty)



# setup a measurement scenario and introduce uncertainties
steps = np.arange(50,2000 + 500,500)
meas_pts = np.array([steps, 
                     np.full(len(steps),0),
                     np.full(len(steps),100)]).T

# meas_pts = np.array([meas_pts[-1]])

meas_pts = meas_pts[-3:]

CT_cfg = {
    'no_scans' : 100000,
    'points' : meas_pts,
    'acq_time' :1, # 1 s
    'max_speed' : 50,    # degree.s^-1
    'max_acc' : 100,     # degree.s^-2
    'sync': True
}

pl_mc.generate_complex_trajectory(['sterenn', 'whittle','koshava'], CT_cfg)
pl_mc.generate_uncertainties('koshava')
pl_mc.generate_uncertainties('whittle')
pl_mc.generate_uncertainties('sterenn')
# pl_mc.generate_complex_trajectory('koshava', CT_cfg)



In [4]:
%time
# create a turbulent flow field
atmo_cfg={'wind_speed':10,
'upward_velocity':0,
'wind_from_direction':45,
'reference_height':100,
'shear_exponent':0.2,
}


pl_mc.generate_flow_field(['sterenn', 'whittle','koshava'], atmo_cfg, "uniform")

# calculate LOS measurements 

pl_mc.project_to_los('sterenn', "power_law")
pl_mc.project_to_los('whittle', "power_law")
pl_mc.project_to_los('koshava', "power_law")


pl_mc.reconstruct_wind(['sterenn', 'whittle','koshava'], 'triple-Doppler')
pl_mc.reconstruct_wind(['sterenn', 'whittle'], 'dual-Doppler', 100)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


ValueError: could not broadcast input array from shape (1000,3) into shape (3000)

In [5]:
pl_mc.data.meas_cfg['sterenn']

{'position': array([   0, -500,    0]),
 'uncertainty': {'unc_az': {'mu': 0, 'std': 0.1, 'units': 'deg'},
  'unc_el': {'mu': 0, 'std': 0.1, 'units': 'deg'},
  'unc_rng': {'mu': 0, 'std': 10, 'units': 'm'},
  'unc_est': {'mu': 0, 'std': 0.1, 'units': 'm.s^-1'},
  'corr_coef': 0},
 'config': {'scan_type': 'CT',
  'max_scn_speed': 50,
  'max_scn_acc': 100,
  'scn_speed': 9.693567925549074,
  'no_los': 3,
  'no_scans': 100000,
  'sectrsz': 11.75638405779182,
  'scn_tm': 4.356656980301047,
  'rtn_tm': 0.9697993218307309,
  'az': array([64.53665494, 72.1213034 , 76.293039  ]),
  'el': array([4.91458736, 3.51358171, 2.71328774]),
  'rng': array([1167.26175299, 1631.71688721, 2112.46301743])}}

In [5]:
tmp = pl_mc.az.values.reshape(100000,3).reshape(20000,5,3).mean(axis = 1)

tmp

array([[64.53665494, 72.1213034 , 76.293039  ],
       [64.53665494, 72.1213034 , 76.293039  ],
       [64.53665494, 72.1213034 , 76.293039  ],
       ...,
       [64.53665494, 72.1213034 , 76.293039  ],
       [64.53665494, 72.1213034 , 76.293039  ],
       [64.53665494, 72.1213034 , 76.293039  ]])

In [10]:
tmp_mean = tmp.mean(axis = 1)
tmp_mean[1]
tmp_mean[0]

array([115.46334506, 107.8786966 , 103.706961  ])

In [111]:
no_scans = 10
no_los = 3
az = pl_mc.az[0]
az_scan_split = np.asarray(np.split(az, no_scans, axis = 0))

az_scan_grouping = np.asarray(np.split(az_scan_split, 5))
az_scan_grouping
# az_scan_group_split
az_scan_grouping.mean(axis = 1).flatten()

array([64.53665494, 72.1213034 , 76.293039  , 64.53665494, 72.1213034 ,
       76.293039  , 64.53665494, 72.1213034 , 76.293039  , 64.53665494,
       72.1213034 , 76.293039  , 64.53665494, 72.1213034 , 76.293039  ])

In [12]:
3*10/5

6.0

In [20]:
def _scan_average(a, no_scans, no_avg):
    
    scan_split = np.asarray(np.split(a, no_scans, axis = 0))
    scan_grouping = np.asarray(np.split(scan_split, int(no_scans/no_avg)))

    return scan_grouping
    

In [21]:
_scan_average(pl_mc.az, 10, 5).mean(axis = 1).flatten()

array([64.53665494, 72.1213034 , 76.293039  , 64.53665494, 72.1213034 ,
       76.293039  ])

In [16]:
az


vrad = np.asarray(np.split(self.data.los[lidar_id].vrad.values,
                            self.data.los[lidar_id].no_scans.values))


array([76.293039, 76.293039, 76.293039, ..., 76.293039, 76.293039,
       76.293039])

In [21]:
# triple-Doppler
los = np.array([pl_mc.data.los['koshava'].vrad, 
                pl_mc.data.los['sterenn'].vrad, 
                pl_mc.data.los['whittle'].vrad]).T


az = np.array([pl_mc.data.los['koshava'].az, 
               pl_mc.data.los['sterenn'].az, 
               pl_mc.data.los['whittle'].az])

el = np.array([pl_mc.data.los['koshava'].el, 
               pl_mc.data.los['sterenn'].el,
               pl_mc.data.los['whittle'].el])

td_rc(los[0], az.T[0], el.T[0], 1)

td_rc_array(los, az, el)

(array([10.]), array([45.]))

In [192]:
# dual-Doppler


los = np.array([pl_mc.data.los['whittle'].vrad, 
                pl_mc.data.los['sterenn'].vrad]).T


az = np.array([pl_mc.data.los['whittle'].az, 
               pl_mc.data.los['sterenn'].az])

el = np.array([pl_mc.data.los['whittle'].el, 
               pl_mc.data.los['sterenn'].el])

dd_rc(los[0], az.T[0], el.T[0], 0)

# dd_rc_array(los, az, el, 1)

(-7.192254182225042, -6.936620630811991, 9.992258303183528, 46.036538710526024)

In [193]:
dd_rc_array(los, az, el)

(array([10.]), array([45.]))

In [185]:
# dual-Doppler
los = np.array([pl_mc.data.los['koshava'].vrad, 
                pl_mc.data.los['sterenn'].vrad]).T


az = np.array([pl_mc.data.los['koshava'].az, 
               pl_mc.data.los['sterenn'].az])

el = np.array([pl_mc.data.los['koshava'].el, 
               pl_mc.data.los['sterenn'].el])

dd_rc(los[0], az.T[0], el.T[0], 0)

(6.427876096865394, 7.660444431189777, 9.999999999999998, 220.00000000000003)

In [182]:
dd_rs_ar(los, az, el, 0)


dd_rc(los[0], az.T[0], el.T[0], 1)

(6.555173129774655, 7.640206795843824, 10.066928759287899, 220.6290565986071)

In [151]:
def dd_rs_ar(los, azimuth, elevation, rc_type = 0):

    azimuth = np.radians(azimuth)
    elevation = np.radians(elevation)
    
    if rc_type == 0:
        R_ws = np.array([[np.sin(azimuth[0]), np.cos(azimuth[0])],
                         [np.sin(azimuth[1]), np.cos(azimuth[1])]]).T
    else:
        R_ws = np.array([[np.sin(azimuth[0])*np.cos(elevation[0]), np.cos(azimuth[0])*np.cos(elevation[0])],
                         [np.sin(azimuth[1])*np.cos(elevation[1]), np.cos(azimuth[1])*np.cos(elevation[1])]]).T

    R_ws_inv = np.linalg.inv(R_ws)


    uvw_array = np.einsum('ijk,ik->ij', R_ws_inv,los)
    V_h_array = np.sqrt(uvw_array[:,0]**2 + uvw_array[:,1]**2)

    wind_dir_array = (90 - np.arctan2(-uvw_array[:,0],-uvw_array[:,1])* (180 / np.pi)) % 360    
    return V_h_array, wind_dir_array

In [139]:
def td_rs_ar(los, azimuth, elevation, rc_type = 0):

    azimuth = np.radians(azimuth)
    elevation = np.radians(elevation)
    
    if rc_type == 0:
        R_ws = np.array([[np.sin(azimuth[0]), np.cos(azimuth[0]), np.sin(elevation[0])],
                         [np.sin(azimuth[1]), np.cos(azimuth[1]), np.sin(elevation[1])],
                         [np.sin(azimuth[2]), np.cos(azimuth[2]), np.sin(elevation[2])]]).T
    else:
        R_ws = np.array([[np.sin(azimuth[0])*np.cos(elevation[0]), np.cos(azimuth[0])*np.cos(elevation[0]), np.sin(elevation[0])],
                         [np.sin(azimuth[1])*np.cos(elevation[1]), np.cos(azimuth[1])*np.cos(elevation[1]), np.sin(elevation[1])],
                         [np.sin(azimuth[2])*np.cos(elevation[2]), np.cos(azimuth[2])*np.cos(elevation[2]), np.sin(elevation[2])]]).T
        
    R_ws_inv = np.linalg.inv(R_ws)


    uvw_array = np.einsum('ijk,ik->ij', R_ws_inv,los)
    V_h_array = np.sqrt(uvw_array[:,0]**2 + uvw_array[:,1]**2)

    wind_dir_array = (90 - np.arctan2(-uvw_array[:,0],-uvw_array[:,1])* (180 / np.pi)) % 360    
    return V_h_array, wind_dir_array

In [15]:
def td_rc(los, azimuth, elevation, rc_type = 0):


    azimuth = np.radians(azimuth)
    elevation = np.radians(elevation)
    
    if rc_type == 0:
        R_ws = np.array([[np.sin(azimuth[0]), np.cos(azimuth[0]), np.sin(elevation[0])],
                         [np.sin(azimuth[1]), np.cos(azimuth[1]), np.sin(elevation[1])],
                         [np.sin(azimuth[2]), np.cos(azimuth[2]), np.sin(elevation[2])]])
    else:
        R_ws = np.array([[np.sin(azimuth[0])*np.cos(elevation[0]), np.cos(azimuth[0])*np.cos(elevation[0]), np.sin(elevation[0])],
                         [np.sin(azimuth[1])*np.cos(elevation[1]), np.cos(azimuth[1])*np.cos(elevation[1]), np.sin(elevation[1])],
                         [np.sin(azimuth[2])*np.cos(elevation[2]), np.cos(azimuth[2])*np.cos(elevation[2]), np.sin(elevation[2])]])

    u, v, w = np.dot(np.linalg.inv(R_ws),los)
#     u, v, w = los.dot(np.linalg.inv(R_ws))    
    wind_speed = np.sqrt(u**2 + v**2)

    wind_direction = (90 - np.degrees(np.arctan2(-v, -u))) % 360

    return u, v, w, wind_speed, wind_direction

In [180]:
def dd_rc(los, azimuth, elevation, rc_type = 0):


    azimuth = np.radians(azimuth)
    elevation = np.radians(elevation)

    if rc_type == 0:
        R_ws = np.array([[np.sin(azimuth[0]), np.cos(azimuth[0])],
                         [np.sin(azimuth[1]), np.cos(azimuth[1])]])
    else:
        R_ws = np.array([[np.sin(azimuth[0])*np.cos(elevation[0]), np.cos(azimuth[0])*np.cos(elevation[0])],
                         [np.sin(azimuth[1])*np.cos(elevation[1]), np.cos(azimuth[1])*np.cos(elevation[1])]])


    u, v = np.dot(np.linalg.inv(R_ws),los)
    wind_speed = np.sqrt(u**2 + v**2)

    wind_direction = (90 - np.degrees(np.arctan2(-v, -u))) % 360

    return u, v, wind_speed, wind_direction

In [142]:
def dd_rs_ar(los, azimuth, elevation, rc_type = 0):

    azimuth = np.radians(azimuth)
    elevation = np.radians(elevation)
    
    if rc_type == 0:
        R_ws = np.array([[np.sin(azimuth[0]), np.cos(azimuth[0])],
                         [np.sin(azimuth[1]), np.cos(azimuth[1])]]).T
    else:
        R_ws = np.array([[np.sin(azimuth[0])*np.cos(elevation[0]), np.cos(azimuth[0])*np.cos(elevation[0])],
                         [np.sin(azimuth[1])*np.cos(elevation[1]), np.cos(azimuth[1])*np.cos(elevation[1])]]).T

    R_ws_inv = np.linalg.inv(R_ws)


    uvw_array = np.einsum('ijk,ik->ij', R_ws_inv,los)
    V_h_array = np.sqrt(uvw_array[:,0]**2 + uvw_array[:,1]**2)

    wind_dir_array = (90 - np.arctan2(-uvw_array[:,0],-uvw_array[:,1])* (180 / np.pi)) % 360    
    return V_h_array, wind_dir_array

In [107]:
            los = u * np.sin(azimuth) * np.cos(elevation) + \
                v * np.cos(azimuth) * np.cos(elevation) + \
                w * np.sin(elevation)


array([90., 90., 90., 90., 90., 90.])

In [94]:
def dd_rc(los, azimuth):

    azimuth = np.radians(azimuth)

    R_ws = np.array([[np.sin(azimuth[0]), np.cos(azimuth[0])],
                     [np.sin(azimuth[1]), np.cos(azimuth[1])]])
    
    R_ws_inv = np.linalg.inv(R_ws)

    u, v = np.dot(np.linalg.inv(R_ws),los)
    wind_speed = np.sqrt(u**2 + v**2)

    wind_direction = (90 - np.degrees(np.arctan2(-v, -u))) % 360

    return u, v, wind_speed, wind_direction

In [54]:
u,v,w, ws, wdir = los_to_wind_array(az, el, los)

In [58]:
    R_ws = np.transpose(np.array([
        [np.cos(elevation_array[0]) * np.sin(azimuth_array[0]),np.cos(elevation_array[1]) * np.sin(azimuth_array[1]), np.cos(elevation_array[2]) * np.sin(azimuth_array[2])], 

         [np.cos(elevation_array[0]) * np.cos(azimuth_array[0]),np.cos(elevation_array[1]) * np.cos(azimuth_array[1]), np.cos(elevation_array[2]) * np.cos(azimuth_array[2])],

         [np.sin(elevation_array[0]),np.sin(elevation_array[1]),np.sin(elevation_array[2])],

    ]))


array([-1.24277122e+15, -1.24277122e+15, -1.24277122e+15, -1.24277122e+15,
       -1.24277122e+15, -1.24277122e+15])

In [39]:
R_ws = np.array([[np.sin(np.radians(pl_mc.data.los['koshava'].az)), np.cos(np.radians(pl_mc.data.los['koshava'].az))],
                 [np.sin(np.radians(pl_mc.data.los['sterenn'].az)), np.cos(np.radians(pl_mc.data.los['sterenn'].az))]])

In [48]:
    R_ws = np.array([[np.sin(az[:, 0]), np.cos(az[:, 0])],
                     [np.sin(az[:, 1]), np.cos(az[:, 1])]])

In [18]:
def los_to_wind_array(azimuth_array, elevation_array, v_los_array):
    R_ws = np.transpose(np.array([
        [np.cos(elevation_array[0]) * np.sin(azimuth_array[0]),np.cos(elevation_array[1]) * np.sin(azimuth_array[1]), np.cos(elevation_array[2]) * np.sin(azimuth_array[2])], 

         [np.cos(elevation_array[0]) * np.cos(azimuth_array[0]),np.cos(elevation_array[1]) * np.cos(azimuth_array[1]), np.cos(elevation_array[2]) * np.cos(azimuth_array[2])],

         [np.sin(elevation_array[0]),np.sin(elevation_array[1]),np.sin(elevation_array[2])],

    ]))

    R_ws_inv = np.linalg.inv(R_ws)
    
    
    uvw_array = np.einsum('ijk,ik->ij', R_ws_inv,v_los_array)
    V_h_array = np.sqrt(uvw_array[:,0]**2 + uvw_array[:,1]**2)
    
    
    wind_dir_array = (90 - np.arctan2(-uvw_array[:,1], -uvw_array[:,0])* (180 / np.pi)) % 360    
    return uvw_array[:,0], uvw_array[:,1],uvw_array[:,2], V_h_array, wind_dir_array

In [51]:
los.shape

(62, 2)

In [52]:
u, v = np.dot(np.linalg.inv(R_ws),los)

LinAlgError: Last 2 dimensions of the array must be square

In [34]:
u,v,ws,wdir = dd_rc(los, az)

ValueError: shapes (2,2,2) and (62,2) not aligned: 2 (dim 2) != 62 (dim 0)

In [177]:
def dd_rc(los, azimuth, elevation, ):
    """Calculates u and v components by using IVAP algo on set of LOS speed
    measurements acquired using PPI scans.

    Parameters
    ----------
    los : numpy
        Array containing LOS speed measurements
    azimuth : numpy
        Array containing azimuth angles corresponding to LOS speed measurements

    Returns
    -------
    u : numpy
        Array of reconstracted u component of the wind
    v : numpy
        Array of reconstracted v component of the wind
    wind_speed: numpy
        Array of reconstracted horizontal wind speed
    """

    azimuth = np.radians(azimuth)

    R_ws = np.array([[np.sin(azimuth[0]), np.cos(azimuth[0])],
                     [np.sin(azimuth[1]), np.cos(azimuth[1])]])

    u, v = np.dot(np.linalg.inv(R_ws),los)
    wind_speed = np.sqrt(u**2 + v**2)

    wind_direction = (90 - np.degrees(np.arctan2(-v, -u))) % 360

    return u, v, wind_speed, wind_direction

In [ ]:
np.arctan2